<a href="https://colab.research.google.com/github/aSafarpoor/AI_Project_BypedalWalker/blob/master/classifier_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#read data

In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
def load_txt_file(filename):
	with open(filename, 'r') as file:
		return [int(line.strip()) for line in file]

def load_txt_file_for_edges(filename):
	with open(filename, 'r') as file:
		return [list(map(int,line.strip().split())) for line in file]

In [68]:
edges = load_txt_file_for_edges('edges.txt')


btrain = load_txt_file('btrain.txt')
strain = load_txt_file('strain.txt')


btest = load_txt_file('btest.txt')
stest = load_txt_file('stest.txt')
nodes = list(set(np.array(edges).reshape(-1)))
num_nodes = len(nodes)

#Transductive

In [7]:
# !pip install torch torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv
!pip install torch
!pip install torch-geometric

In [130]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch_geometric.utils as pyg_utils
from torch_geometric.nn import GATConv
from sklearn.metrics import accuracy_score

from torch_geometric.utils import subgraph

In [125]:
class GCN_ETH(torch.nn.Module):
    def __init__(self, num_node_features, num_layers, hidden_width, dropout = True):
        super().__init__()

        self.dropout = dropout
        self.convs = torch.nn.ModuleList()
        input_width = num_node_features
        self.num_classes = 2
        for i in range(num_layers):
            if i == 0:
                self.convs.append(GCNConv(input_width, hidden_width, bias=False))
            elif i == num_layers - 1:
                self.convs.append(GCNConv(hidden_width, self.num_classes, bias=False))
            else:
                self.convs.append(GCNConv(hidden_width, hidden_width, bias=False))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        h = x
        i = 0
        for conv in self.convs:
            if self.dropout:
                h = F.dropout(h, p=0.5, training=self.training)
            h = conv(h, edge_index)
            if i < len(self.convs) - 1:
                h = F.tanh(h)
            i += 1

        return F.log_softmax(h, dim=1)


In [126]:
class GAT_ETH(torch.nn.Module):
    def __init__(self, input_width, num_layers, hidden_width, num_classes, num_heads, dropout: bool = True):
        super().__init__()
        self.dropout = dropout
        self.num_classes = num_classes
        self.convs = torch.nn.ModuleList()

        for i in range(num_layers):
            if i == 0:  # First layer
                self.convs.append(GATConv(input_width, hidden_width, heads=num_heads))
            elif i == num_layers - 1:  # Last layer
                self.convs.append(GATConv(hidden_width * num_heads, self.num_classes, heads=1))
            else:  # Middle layers
                self.convs.append(GATConv(hidden_width * num_heads, hidden_width, heads=num_heads))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # x = self.conv1(x, edge_index)
        # # x = F.elu(x)
        # x = F.tanh(x)
        # x = self.conv2(x, edge_index)
        # h = x

        h = x
        i = 0
        for conv in self.convs:
            if self.dropout:
                h = F.dropout(h, p=0.5, training=self.training)
            h = conv(h, edge_index)
            if i < len(self.convs) - 1:
                h = F.tanh(h)
                # h = F.elu(h)
            i += 1

        return F.log_softmax(h, dim=1)



In [105]:
class GCN1(torch.nn.Module):
    def __init__(self, num_node_features, hidden_dim, num_classes):
        super(GCN1, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)


    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [106]:
class GCN2(torch.nn.Module):
    def __init__(self, num_node_features, hidden_dim1, hidden_dim2, num_classes):
        super(GCN2, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_dim1)
        self.conv2 = GCNConv(hidden_dim1, hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x = self.conv2(x, edge_index)
        x = F.relu(x)

        x = self.conv3(x, edge_index)
        x = F.softmax(x,dim=1)
        return F.log_softmax(x, dim=1)


In [107]:
class GAT1(torch.nn.Module):
    def __init__(self, num_node_features, hidden_dim1, hidden_dim2, num_classes, num_heads=1):
        super(GAT1, self).__init__()
        self.gat1 = GATConv(num_node_features, hidden_dim1, heads=num_heads, concat=True)
        self.gat2 = GATConv(hidden_dim1 * num_heads, hidden_dim2, heads=num_heads, concat=True)
        self.gat3 = GATConv(hidden_dim2 * num_heads, num_classes, heads=1, concat=False)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gat1(x, edge_index)
        x = F.relu(x)

        x = self.gat2(x, edge_index)
        x = F.relu(x)

        x = self.gat3(x, edge_index)

        return F.log_softmax(x, dim=1)



In [108]:
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(1, 16)  # 1 input feature, 16 output features
        self.conv2 = GCNConv(16, 2)  # 16 input features, 2 output features (binary classification)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x



In [109]:
def main_transductive(model):

    edge_index = torch.tensor([[e[0] for e in edges], [e[1] for e in edges]], dtype=torch.long)  # Replace with actual edges


    # Assign labels (0 for benign, 1 for sybil)
    labels = torch.tensor([0 if i in btrain or i in btest else 1 for i in range(num_nodes)])

    # Initialize the node features
    x = torch.zeros((num_nodes, 1))  # 1-dimensional embeddings, all zeros initially

    # Assign initial embeddings based on the conditions
    for node in btrain:
        x[node] = 0  # Benign training nodes
    for node in strain:
        x[node] = 1  # Sybil training nodes
    for node in btest + stest:
        x[node] = 0.5  # Test nodes

    # Create the PyTorch Geometric data object
    data = Data(x=x, edge_index=edge_index, y=labels)

    # Define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    loss_fn = torch.nn.CrossEntropyLoss()





    # Split data into training and testing masks
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    # Assign masks for training and testing nodes
    train_mask[btrain + strain] = True
    test_mask[btest + stest] = True

    # Training loop
    def train():
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = loss_fn(out[train_mask], data.y[train_mask])  # Only consider training nodes for loss
        loss.backward()
        optimizer.step()
        return loss.item()

    # Testing function
    def test():
        model.eval()
        out = model(data)
        pred = out.argmax(dim=1)  # Get predictions
        train_acc = accuracy_score(data.y[train_mask].cpu(), pred[train_mask].cpu())
        test_acc = accuracy_score(data.y[test_mask].cpu(), pred[test_mask].cpu())
        return train_acc, test_acc

    # Training the model
    epochs = 100
    for epoch in range(epochs):
        loss = train()
        train_acc, test_acc = test()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

    # Final evaluation
    train_acc, test_acc = test()
    print(f'Final Train Accuracy: {train_acc:.4f}, Final Test Accuracy: {test_acc:.4f}')


In [110]:
main_transductive(GCN())

Epoch 0, Loss: 0.6965, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 10, Loss: 0.6818, Train Acc: 0.8938, Test Acc: 0.8529
Epoch 20, Loss: 0.6675, Train Acc: 0.9125, Test Acc: 0.8359
Epoch 30, Loss: 0.6428, Train Acc: 0.8938, Test Acc: 0.8514
Epoch 40, Loss: 0.6031, Train Acc: 0.8938, Test Acc: 0.8529
Epoch 50, Loss: 0.5481, Train Acc: 0.9062, Test Acc: 0.8622
Epoch 60, Loss: 0.4827, Train Acc: 0.9125, Test Acc: 0.8746
Epoch 70, Loss: 0.4130, Train Acc: 0.9250, Test Acc: 0.8839
Epoch 80, Loss: 0.3395, Train Acc: 0.9313, Test Acc: 0.8978
Epoch 90, Loss: 0.2675, Train Acc: 0.9750, Test Acc: 0.9226
Final Train Accuracy: 0.9812, Final Test Accuracy: 0.9551


In [111]:
main_transductive(GCN1(num_node_features=1, hidden_dim=16, num_classes=2))

Epoch 0, Loss: 0.6669, Train Acc: 0.7125, Test Acc: 0.6285
Epoch 10, Loss: 0.6197, Train Acc: 0.9500, Test Acc: 0.9102
Epoch 20, Loss: 0.5725, Train Acc: 0.9500, Test Acc: 0.9056
Epoch 30, Loss: 0.5173, Train Acc: 0.9563, Test Acc: 0.9102
Epoch 40, Loss: 0.4499, Train Acc: 0.9750, Test Acc: 0.9288
Epoch 50, Loss: 0.3727, Train Acc: 0.9812, Test Acc: 0.9582
Epoch 60, Loss: 0.2958, Train Acc: 0.9875, Test Acc: 0.9768
Epoch 70, Loss: 0.2297, Train Acc: 0.9875, Test Acc: 0.9768
Epoch 80, Loss: 0.1791, Train Acc: 0.9812, Test Acc: 0.9799
Epoch 90, Loss: 0.1433, Train Acc: 0.9812, Test Acc: 0.9783
Final Train Accuracy: 0.9812, Final Test Accuracy: 0.9783


In [112]:
main_transductive(GCN2(num_node_features=1, hidden_dim1=16, hidden_dim2 = 16, num_classes=2))

Epoch 0, Loss: 0.6828, Train Acc: 0.6937, Test Acc: 0.6780
Epoch 10, Loss: 0.6512, Train Acc: 0.8625, Test Acc: 0.8437
Epoch 20, Loss: 0.5903, Train Acc: 0.9062, Test Acc: 0.8762
Epoch 30, Loss: 0.5077, Train Acc: 0.9250, Test Acc: 0.9009
Epoch 40, Loss: 0.4406, Train Acc: 0.9313, Test Acc: 0.9149
Epoch 50, Loss: 0.4064, Train Acc: 0.9437, Test Acc: 0.9195
Epoch 60, Loss: 0.3908, Train Acc: 0.9437, Test Acc: 0.9288
Epoch 70, Loss: 0.3832, Train Acc: 0.9437, Test Acc: 0.9303
Epoch 80, Loss: 0.3789, Train Acc: 0.9500, Test Acc: 0.9334
Epoch 90, Loss: 0.3766, Train Acc: 0.9500, Test Acc: 0.9334
Final Train Accuracy: 0.9500, Final Test Accuracy: 0.9350


In [127]:
main_transductive(GCN_ETH(num_node_features=1, num_layers=3, hidden_width=16))

Epoch 0, Loss: 0.6791, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 10, Loss: 0.6516, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 20, Loss: 0.6497, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 30, Loss: 0.6540, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 40, Loss: 0.6470, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 50, Loss: 0.6540, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 60, Loss: 0.6494, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 70, Loss: 0.6529, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 80, Loss: 0.6550, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 90, Loss: 0.6525, Train Acc: 0.5000, Test Acc: 0.5000
Final Train Accuracy: 0.5000, Final Test Accuracy: 0.5000


In [113]:
main_transductive(GAT1(num_node_features=1, hidden_dim1=16, hidden_dim2=16, num_classes=2, num_heads=8))

Epoch 0, Loss: 0.6904, Train Acc: 0.5125, Test Acc: 0.5015
Epoch 10, Loss: 0.4217, Train Acc: 0.9187, Test Acc: 0.9195
Epoch 20, Loss: 0.1939, Train Acc: 0.9688, Test Acc: 0.9489
Epoch 30, Loss: 0.1070, Train Acc: 0.9812, Test Acc: 0.9659
Epoch 40, Loss: 0.0575, Train Acc: 0.9875, Test Acc: 0.9783
Epoch 50, Loss: 0.0277, Train Acc: 0.9938, Test Acc: 0.9845
Epoch 60, Loss: 0.0083, Train Acc: 0.9938, Test Acc: 0.9892
Epoch 70, Loss: 0.0045, Train Acc: 1.0000, Test Acc: 0.9845
Epoch 80, Loss: 0.0034, Train Acc: 1.0000, Test Acc: 0.9861
Epoch 90, Loss: 0.0030, Train Acc: 1.0000, Test Acc: 0.9876
Final Train Accuracy: 1.0000, Final Test Accuracy: 0.9861


In [122]:
main_transductive(GAT_ETH(input_width=1, num_layers=4, hidden_width=16, num_classes=2, num_heads=8, dropout= True))

Epoch 0, Loss: 0.7009, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 10, Loss: 0.6382, Train Acc: 0.6188, Test Acc: 0.5929
Epoch 20, Loss: 0.3546, Train Acc: 0.9625, Test Acc: 0.9474
Epoch 30, Loss: 0.3648, Train Acc: 1.0000, Test Acc: 0.9969
Epoch 40, Loss: 0.3178, Train Acc: 1.0000, Test Acc: 0.9969
Epoch 50, Loss: 0.3569, Train Acc: 0.9187, Test Acc: 0.8932
Epoch 60, Loss: 0.3195, Train Acc: 0.8375, Test Acc: 0.7771
Epoch 70, Loss: 0.3601, Train Acc: 0.9875, Test Acc: 0.9628
Epoch 80, Loss: 0.3133, Train Acc: 1.0000, Test Acc: 0.9938
Epoch 90, Loss: 0.3191, Train Acc: 0.9125, Test Acc: 0.8545
Final Train Accuracy: 0.9875, Final Test Accuracy: 0.9737


In [123]:
main_transductive(GAT1(num_node_features=1, hidden_dim1=4, hidden_dim2=4, num_classes=2, num_heads=8))

Epoch 0, Loss: 0.6862, Train Acc: 0.8438, Test Acc: 0.8204
Epoch 10, Loss: 0.5923, Train Acc: 0.9125, Test Acc: 0.9009
Epoch 20, Loss: 0.4325, Train Acc: 0.9313, Test Acc: 0.9272
Epoch 30, Loss: 0.2460, Train Acc: 0.9625, Test Acc: 0.9520
Epoch 40, Loss: 0.1311, Train Acc: 0.9750, Test Acc: 0.9582
Epoch 50, Loss: 0.0909, Train Acc: 0.9875, Test Acc: 0.9582
Epoch 60, Loss: 0.0774, Train Acc: 0.9875, Test Acc: 0.9613
Epoch 70, Loss: 0.0649, Train Acc: 0.9875, Test Acc: 0.9659
Epoch 80, Loss: 0.0530, Train Acc: 0.9875, Test Acc: 0.9752
Epoch 90, Loss: 0.0406, Train Acc: 0.9875, Test Acc: 0.9752
Final Train Accuracy: 0.9938, Final Test Accuracy: 0.9845


In [124]:
main_transductive(GCN2(num_node_features=1, hidden_dim1=6, hidden_dim2 = 6, num_classes=2))

Epoch 0, Loss: 0.6912, Train Acc: 0.8000, Test Acc: 0.7771
Epoch 10, Loss: 0.6836, Train Acc: 0.9313, Test Acc: 0.9071
Epoch 20, Loss: 0.6699, Train Acc: 0.9375, Test Acc: 0.9025
Epoch 30, Loss: 0.6475, Train Acc: 0.9062, Test Acc: 0.8777
Epoch 40, Loss: 0.6152, Train Acc: 0.9125, Test Acc: 0.8793
Epoch 50, Loss: 0.5750, Train Acc: 0.9062, Test Acc: 0.8839
Epoch 60, Loss: 0.5299, Train Acc: 0.9313, Test Acc: 0.8947
Epoch 70, Loss: 0.4854, Train Acc: 0.9375, Test Acc: 0.9087
Epoch 80, Loss: 0.4497, Train Acc: 0.9375, Test Acc: 0.9149
Epoch 90, Loss: 0.4253, Train Acc: 0.9375, Test Acc: 0.9164
Final Train Accuracy: 0.9375, Final Test Accuracy: 0.9226


#inductive

In [144]:
import random

def create_subgraph(data, nodes_subset):

    # Convert nodes_subset to tensor
    nodes_subset = torch.tensor(nodes_subset, dtype=torch.long)

    # Create a subgraph with only the edges between nodes in nodes_subset
    edge_index, edge_attr = subgraph(nodes_subset, data.edge_index, relabel_nodes=True)

    # Select the corresponding node features
    x = data.x[nodes_subset]

    # Select the corresponding labels (if any)
    y = data.y[nodes_subset] if data.y is not None else None

    # Return the subgraph
    subgraph_data = Data(x=x, edge_index=edge_index, y=y)

    return subgraph_data


def main_inductive(model):

    edge_index = torch.tensor([[e[0] for e in edges], [e[1] for e in edges]], dtype=torch.long)  # Replace with actual edges

    # Define the set of unknown nodes (nodes not in btrain, strain, btest, stest)
    known_nodes = set(btrain + strain + btest + stest)
    unknown_nodes = list(set(range(num_nodes)) - known_nodes)

    # Sample 1/4 of unknown nodes for TRAIN
    random.shuffle(unknown_nodes)
    unknown_sample_for_train = unknown_nodes[:len(unknown_nodes)//4]

    # Create subgraph TRAIN consisting of btrain, strain, and 1/4 of unknown nodes
    train_nodes = btrain + strain + unknown_sample_for_train

    # Create subgraph TEST consisting of the remaining nodes
    test_nodes = list(set(range(num_nodes)) - set(train_nodes))

    # Assign labels (0 for benign, 1 for sybil)
    labels = torch.tensor([0 if i in btrain or i in btest else 1 for i in range(num_nodes)])

    # Initialize the node features
    x = torch.zeros((num_nodes, 1))  # 1-dimensional embeddings, all zeros initially

    # Assign initial embeddings based on the conditions
    for node in btrain:
        x[node] = 0  # Benign training nodes
    for node in strain:
        x[node] = 1  # Sybil training nodes
    for node in btest + stest:
        x[node] = 0.5  # Test nodes

    # Create the PyTorch Geometric data object
    data = Data(x=x, edge_index=edge_index, y=labels)

    # Create subgraphs for training and testing
    train_subgraph = create_subgraph(data, train_nodes)
    test_subgraph = create_subgraph(data, test_nodes)

    # Define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    loss_fn = torch.nn.CrossEntropyLoss()

    # Training loop
    def train():
        model.train()
        optimizer.zero_grad()
        out = model(train_subgraph)
        loss = loss_fn(out, train_subgraph.y)  # Only consider training nodes for loss
        loss.backward()
        optimizer.step()
        return loss.item()

    # Testing function
    def test():
        model.eval()
        out = model(test_subgraph)
        pred = out.argmax(dim=1)  # Get predictions
        test_acc = accuracy_score(test_subgraph.y.cpu(), pred.cpu())
        return test_acc

    # Training the model
    epochs = 500
    for epoch in range(epochs):
        loss = train()
        test_acc = test()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss:.4f}, Test Acc: {test_acc:.4f}')

    # Final evaluation
    test_acc = test()
    print(f'Final Test Accuracy: {test_acc:.4f}')




In [145]:
main_inductive(GCN())

Epoch 0, Loss: 0.7238, Test Acc: 0.5153
Epoch 10, Loss: 0.5275, Test Acc: 0.9042
Epoch 20, Loss: 0.3575, Test Acc: 0.9042
Epoch 30, Loss: 0.2775, Test Acc: 0.9042
Epoch 40, Loss: 0.2726, Test Acc: 0.9042
Epoch 50, Loss: 0.2744, Test Acc: 0.9042
Epoch 60, Loss: 0.2713, Test Acc: 0.9042
Epoch 70, Loss: 0.2705, Test Acc: 0.9042
Epoch 80, Loss: 0.2706, Test Acc: 0.9042
Epoch 90, Loss: 0.2704, Test Acc: 0.9042
Epoch 100, Loss: 0.2703, Test Acc: 0.9042
Epoch 110, Loss: 0.2703, Test Acc: 0.9042
Epoch 120, Loss: 0.2702, Test Acc: 0.9042
Epoch 130, Loss: 0.2701, Test Acc: 0.9042
Epoch 140, Loss: 0.2701, Test Acc: 0.9042
Epoch 150, Loss: 0.2700, Test Acc: 0.9042
Epoch 160, Loss: 0.2700, Test Acc: 0.9042
Epoch 170, Loss: 0.2699, Test Acc: 0.9042
Epoch 180, Loss: 0.2698, Test Acc: 0.9042
Epoch 190, Loss: 0.2698, Test Acc: 0.9042
Epoch 200, Loss: 0.2697, Test Acc: 0.9042
Epoch 210, Loss: 0.2696, Test Acc: 0.9042
Epoch 220, Loss: 0.2696, Test Acc: 0.9042
Epoch 230, Loss: 0.2695, Test Acc: 0.9042
Epo

In [146]:
main_inductive(GAT1(num_node_features=1, hidden_dim1=4, hidden_dim2=4, num_classes=2, num_heads=8))

Epoch 0, Loss: 0.7261, Test Acc: 0.8936
Epoch 10, Loss: 0.3876, Test Acc: 0.9042
Epoch 20, Loss: 0.2960, Test Acc: 0.9042
Epoch 30, Loss: 0.2693, Test Acc: 0.9042
Epoch 40, Loss: 0.2702, Test Acc: 0.9042
Epoch 50, Loss: 0.2668, Test Acc: 0.9042
Epoch 60, Loss: 0.2658, Test Acc: 0.9042
Epoch 70, Loss: 0.2655, Test Acc: 0.9042
Epoch 80, Loss: 0.2654, Test Acc: 0.9042
Epoch 90, Loss: 0.2653, Test Acc: 0.9042
Epoch 100, Loss: 0.2652, Test Acc: 0.9042
Epoch 110, Loss: 0.2652, Test Acc: 0.9042
Epoch 120, Loss: 0.2651, Test Acc: 0.9042
Epoch 130, Loss: 0.2651, Test Acc: 0.9042
Epoch 140, Loss: 0.2651, Test Acc: 0.9042
Epoch 150, Loss: 0.2651, Test Acc: 0.9042
Epoch 160, Loss: 0.2651, Test Acc: 0.9042
Epoch 170, Loss: 0.2651, Test Acc: 0.9042
Epoch 180, Loss: 0.2651, Test Acc: 0.9042
Epoch 190, Loss: 0.2651, Test Acc: 0.9042
Epoch 200, Loss: 0.2651, Test Acc: 0.9042
Epoch 210, Loss: 0.2651, Test Acc: 0.9042
Epoch 220, Loss: 0.2651, Test Acc: 0.9042
Epoch 230, Loss: 0.2651, Test Acc: 0.9042
Epo